## 1. Setup and Configuration

# Install necessary libraries (if not already installed)
!pip install -q plotly pandas

import json
import pandas as pd
import plotly.express as px
import sys
import os

# Add project root to path to import modules
sys.path.append(os.path.abspath('..'))

# Mock agents and tools for this demo
from agents.test_diagnostics_agent import TestDiagnosticsAgent
from agents.root_cause_agent import RootCauseAnalyzerAgent
from agents.action_planner_agent import ActionPlannerAgent
from utils.memory_handler import read_memory, update_memory, clear_memory

## 2. Simulate a CI/CD Log Failure

We'll use a sample log file that contains a typical failure from a Jenkins pipeline.

with open('../data/sample_logs/jenkins_failure.log', 'r') as f:
    ci_logs = f.read()

print("--- Sample CI Log ---")
print(ci_logs)

## 3. Run the Orchestration Pipeline

def run_pipeline(logs):
    """Simulates the agent orchestration process."""
    print("\n--- Starting Pipeline ---")
    
    # 1. Test Diagnostics Agent
    diagnostics_agent = TestDiagnosticsAgent(name="TestDiagnostics")
    diag_result = diagnostics_agent.process(logs)
    print(f"\n[TestDiagnosticsAgent] -> {diag_result['summary']}")
    update_memory(diag_result['failed_tests'])

    # 2. Root Cause Analyzer Agent
    root_cause_agent = RootCauseAnalyzerAgent(name="RootCauseAnalyzer")
    rca_result = root_cause_agent.process(diag_result)
    print(f"\n[RootCauseAnalyzerAgent] -> {rca_result['summary']}")
    
    # 3. Action Planner Agent
    action_planner_agent = ActionPlannerAgent(name="ActionPlanner")
    plan = action_planner_agent.process(rca_result)
    print(f"\n[ActionPlannerAgent] -> {plan['summary']}")
    
    print("\n--- Pipeline Finished ---")
    return plan

final_plan = run_pipeline(ci_logs)

## 4. Visualize Issue Frequency

The orchestrator maintains a `memory_bank.json` to track recurring issues. Let's visualize the failure counts.

def visualize_memory():
    memory = read_memory()
    if not memory:
        print("Memory bank is empty.")
        return
    
    df = pd.DataFrame(list(memory.items()), columns=['Test', 'Failure Count'])
    df = df.sort_values('Failure Count', ascending=False)
    
    fig = px.bar(df, 
                 x='Test', 
                 y='Failure Count', 
                 title='Flaky Test Failure Frequency',
                 labels={'Test': 'Test Case', 'Failure Count': 'Number of Failures'},
                 template='plotly_dark')
    
    fig.update_layout(xaxis_tickangle=-45)
    fig.show()

visualize_memory()

## 5. Clear Memory

You can clear the memory to reset the state for a new simulation.

# clear_memory()
# print("Memory bank has been cleared.")